In [1]:
import os
working_directory = 'C:/Users/MAngO/Dropbox/UST/Year I/Yuany CSIC/Project'
os.chdir(working_directory)

In [2]:
import jieba
with open('dream_figures.txt', 'rb') as f:
    characters_names = [line.decode('utf-8').strip('\n') for line in f.readlines()]
for name in characters_names:
    jieba.add_word(name)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MAngO\AppData\Local\Temp\jieba.cache
Loading model cost 0.937 seconds.
Prefix dict has been built succesfully.


In [3]:
# Training of word2vec
import gensim
with open('dream.txt', 'rb') as f:
    data = [line.decode('utf-8').strip() for line in f.readlines()if line.decode('utf-8').strip()]

sentences = []
for line in data:
    words = list(jieba.cut(line))
    sentences.append(words)
    
# By default setting, we train word2vec with CBOW and Negative-Sampling, words with count less than 10 are removed
model = gensim.models.Word2Vec(sentences,size=100,window=5,min_count=10)

C:\Users\MAngO\Anaconda3\envs\tf\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [4]:
model['宝玉']

array([-0.08804332, -0.53033477, -0.66613024, -0.1551435 , -0.35534421,
       -0.03870843, -0.21183251,  0.58766025, -0.25875148, -0.84310699,
       -0.73783082, -0.55144173,  0.05687124, -1.66058624,  1.37546086,
       -0.57371366, -0.14626463, -0.34558025, -0.38663989, -0.1301972 ,
        0.17591901, -0.50929439,  0.08011577,  0.42731753, -0.23153152,
        0.38623524,  0.15924992,  0.71490836,  0.20342124, -0.23314959,
        1.15066624, -0.87204641,  0.66758949,  0.78183824, -0.78160524,
        0.07213362,  0.05746133,  0.33127135,  0.16983888, -0.01203949,
        0.42158478, -0.66407669,  0.33799288, -0.38730383,  0.63126647,
        0.380124  ,  0.11783867,  0.28056723,  1.1385392 ,  0.67765635,
        0.38157699,  0.81880951, -0.66098022,  0.30383947, -0.98767942,
        0.25376529,  0.60259086, -0.02757449,  0.6801005 ,  0.18785532,
        0.50834233,  0.1595711 ,  0.97904354, -0.05315511,  0.01344215,
       -0.44681928, -0.30663002, -0.42049739, -0.19419238, -0.60

In [5]:
# Use gensim built-in function to test the most relevant figures with given relationship: (sum(positive) - negative)
model.most_similar(positive=['王夫人', '凤姐'], negative=[ '薛姨妈'])

[('贾琏', 0.9161045551300049),
 ('贾母', 0.8976159691810608),
 ('尤氏', 0.8508528470993042),
 ('凤姐儿', 0.8405201435089111),
 ('麝月', 0.8253404498100281),
 ('平儿', 0.825271725654602),
 ('贾芸', 0.8228203058242798),
 ('袭人', 0.8221731185913086),
 ('鸳鸯', 0.8218496441841125),
 ('刘姥姥', 0.8152858018875122)]

In [6]:
model.most_similar(positive=['宝玉', '宝钗'], negative=[ '黛玉'])

[('袭人', 0.8978891372680664),
 ('凤姐', 0.855692982673645),
 ('林黛玉', 0.8433758020401001),
 ('紫鹃', 0.8423019647598267),
 ('晴雯', 0.8416805863380432),
 ('贾琏', 0.8381054997444153),
 ('薛姨妈', 0.8275371789932251),
 ('香菱', 0.8089209794998169),
 ('探春', 0.806232213973999),
 ('王夫人', 0.7977675199508667)]

In [7]:
# Load character names and corresponding word vectors to do clustering in next step
import numpy as np
all_names = []

word_vectors = None
np_names = None
for name in characters_names:
    if name in model:
        all_names.append(name)
for name in all_names:
    if word_vectors is None:
        word_vectors = model[name]
    else:
        word_vectors = np.vstack((word_vectors, model[name]))
        np_names = np.array(all_names)

In [8]:
# Use KMeans to cluster character names with their word vectors
import scipy
from sklearn.cluster import KMeans
def kmeans(np_names, word_vectors, N):
    label = KMeans(N).fit(word_vectors).labels_
    n = scipy.stats.mode(label).mode

    remain_names = np_names[label != n]
    remain_vectors = word_vectors[label != n]
    remain_label = KMeans(N).fit(remain_vectors).labels_

    for i in range(N):
        print("类别{}：".format(i + 1))
        for idx, name in enumerate(remain_names[remain_label == i]):
            print(name, end=", ")
            if idx % 10 == 9:
                print('')
        print('')

In [9]:
kmeans(np_names, word_vectors, 3)

类别1：
贾赦, 贾政, 贾珍, 贾母, 邢夫人, 尤氏, 
类别2：
黛玉, 宝钗, 贾琏, 宝玉, 王夫人, 凤姐, 薛姨妈, 李纨, 袭人, 紫鹃, 
鸳鸯, 平儿, 
类别3：
贾环, 贾蓉, 贾芸, 林黛玉, 香菱, 刘姥姥, 晴雯, 麝月, 秋纹, 雪雁, 
琥珀, 莺儿, 芳官, 周瑞家的, 


In [10]:
kmeans(np_names, word_vectors, 5)

类别1：
贾芸, 林黛玉, 香菱, 刘姥姥, 晴雯, 麝月, 紫鹃, 鸳鸯, 平儿, 周瑞家的, 

类别2：
贾环, 贾瑞, 贾蓉, 贾兰, 贾蔷, 史湘云, 薛蟠, 薛蝌, 妙玉, 赵姨娘, 
秋纹, 司棋, 雪雁, 琥珀, 莺儿, 丰儿, 彩云, 翠缕, 宝蟾, 茗烟, 
焙茗, 李贵, 兴儿, 芳官, 秦钟, 冯紫英, 赖大, 林之孝, 林之孝家的, 尤二姐, 

类别3：
贾琏, 贾母, 王夫人, 凤姐, 薛姨妈, 李纨, 
类别4：
贾赦, 贾政, 贾珍, 邢夫人, 尤氏, 
类别5：
黛玉, 宝钗, 宝玉, 袭人, 
